In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Required Libraries

In [ ]:
import os

import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

import tqdm
import nltk
import re
import io
import string
import emoji
import gensim

import transformers
from transformers import pipeline


from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

from IPython.display import Markdown

def bold(string):
    display(Markdown("**" + string + "**"))

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Load Data

In [ ]:
df = pd.read_csv("/kaggle/input/linkedin-canada-data-science-jobs-2024/linkedin_canada.csv")
df.head()

In [ ]:
def df_stats(data):
    bold(" SHAPE ".center(50, "#"))
    print("ROWS: {}".format(data.shape[0]))
    print("COLS: {}".format(data.shape[1]))
    bold(" TYPES ".center(50, "#"))
    print(data.dtypes)
    bold(" MISSING VALUES ".center(50, "#"))
    print(data.isnull().sum())
    bold(" DUPLICATED VALUES ".center(50, "#"))
    print("NUMBER OF DUPLICATED VALUES: {}".format(data.duplicated().sum()))
    #bold(" DESCRIBE ".center(50, "#"))
    #print(data.describe().T)
    bold(" MEMORY USAGE ".center(50, "#"))
    buf = io.StringIO()
    data.info(buf=buf)
    info = buf.getvalue().split("\n")[-2].split(":")[1].strip()
    print("Memory Usage: {}".format(info))

In [ ]:
df_stats(df)

# Preprocess

In [ ]:
df = df.drop(["companyId", "salary"], axis=1)

In [ ]:
def fix_1(text):
    if text == "Over 200 applicants":
        return "200+"
    elif text.startswith("Be among the first 25"):
        return "=25"
    else:
        text = text.replace(" applicants", "")
        return text

In [ ]:
df["applicationsCount"] = df["applicationsCount"].apply(fix_1)

In [ ]:
df.fillna("Unknown", inplace=True)

In [ ]:
df["publishedAt"] = df["publishedAt"].apply(lambda x: x[:7])

# EDA

In [ ]:
df["applicationsCount"].value_counts().head(15).plot(kind="bar")

In [ ]:
df["experienceLevel"].value_counts().head(15).plot(kind="bar")

In [ ]:
df["companyName"].value_counts().head(15).plot(kind="bar")

In [ ]:
df["publishedAt"].value_counts().head(15).plot(kind="bar")

In [ ]:
df["sector"].value_counts().head(15).plot(kind="barh")

In [ ]:
df["title"].value_counts().head(15).plot(kind="barh")

In [ ]:
df["workType"].value_counts().head(15).plot(kind="barh")

In [ ]:
def count_ngrams(corpus, ngram, n):
    vec = CountVectorizer(ngram_range=(ngram,ngram), stop_words="english").fit(corpus)
    bow = vec.transform(corpus).sum(axis=0)
    words_freq = sorted([(word, bow[0, idx]) for word, idx in vec.vocabulary_.items()], key=lambda x: x[1], reverse=True)[:n]
    return words_freq

In [ ]:
def plot_ngrams(ngram_df, ngram_name):
    plt.figure(figsize=(12, 6))
    plt.bar(data=ngram_df, x="Text", height="Count")
    plt.xticks(rotation=90)
    plt.xlabel(ngram_name)
    plt.ylabel("Count")
    plt.title(ngram_name)
    plt.show()

In [ ]:
unigrams = count_ngrams(df["description"], 1, 30)
top_unigram = pd.DataFrame(unigrams, columns=['Text', "Count"])
top_unigram.head()

In [ ]:
plot_ngrams(top_unigram, "Unigrams")

In [ ]:
bigrams = count_ngrams(df["description"], 2, 30)
top_bigram = pd.DataFrame(bigrams, columns=['Text', "Count"])
top_bigram.head()

In [ ]:
plot_ngrams(top_bigram, "Bigrams")

In [ ]:
trigrams = count_ngrams(df["description"], 3, 30)
top_trigram = pd.DataFrame(trigrams, columns=['Text', "Count"])
top_trigram.head()

In [ ]:
plot_ngrams(top_trigram, "Trigrams")

# Zero Shot Classification

In [ ]:
classifier = pipeline("zero-shot-classification")

In [ ]:
candidate_labels = [
    "Machine Learning Engineer",
    "Data Scientist",
    "Software Engineer",
    "Intern",
    "Full Stack Developer",
    "Blockchain Developer",
    "Security Engineer",
    "Frontend Developer",
    "Backend Developer"
]

In [ ]:
results = []
for statement in tqdm.tqdm(df['description'][:50]):
    result = classifier(statement, candidate_labels=candidate_labels)
    results.append(result)

In [ ]:
results_df = df[["description", "title"]][:50]

In [ ]:
results_df["predicted"] = [result["labels"][0] for result in results]

In [ ]:
results_df.sample(10)